In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Define classes and functions
class Time_Series_Dataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        y = self.outputs[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Convolutional LSTM Model
class ConvLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_layers, output_size, num_filters, kernel_size):
        super(ConvLSTM, self).__init__()
        self.hidden_size = len(hidden_sizes)
        self.num_layers = num_layers

        # Convolutional layer
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size)
        
        self.lstms = nn.ModuleList()
        self.lstms.append(nn.LSTM(num_filters, hidden_sizes[0], batch_first=True, bidirectional=True))

        for i in range(1, num_layers):
            self.lstms.append(nn.LSTM(hidden_sizes[i-1]*2, hidden_sizes[i], batch_first=True, bidirectional=True))
        
        self.fc1 = nn.Linear(hidden_sizes[-1] * 2, 20)  # Fully connected layer
        self.fc2 = nn.Linear(20, output_size)

    def forward(self, x):
        # Convolutional layer expects input of shape (batch_size, in_channels, seq_length)
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        x = x.permute(0, 2, 1)  # Convert back to (batch_size, seq_length, num_filters)
        
        h = x
        for lstm in self.lstms:
            out, _ = lstm(h)
            h = out
        out = torch.relu(self.fc1(out[:, -1, :]))
        out = self.fc2(out)
        return out

def split_data(data, input_size, output_size, train_ratio, seed):
    X, y = [], [] 
    total_size = input_size + output_size
    for i in range(len(data) - total_size + 1):
        X.append(features[i:i + input_size])                            
        y.append(target[i + input_size:i + total_size])        
    # 2. shuffle batches and split into train/test
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size = train_ratio, random_state=seed)
    
    return X_train, X_test, Y_train, Y_test

def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_targets = []
    
    with torch.no_grad():  # Disable gradient calculation
        for inputs, targets in dataloader:
            outputs = model(inputs)  # Forward pass
            all_preds.append(outputs)
            all_targets.append(targets)
    
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    
    return all_preds, all_targets

pm = "\u00B1"

Ethereum = pd.read_csv('data/coin_Ethereum.csv')
data = Ethereum.iloc[:, 4:]
features = data[['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']]
features = MinMaxScaler().fit_transform(features) # normalise input
target = data['Close']
target_reshaped = np.array(target).reshape(-1, 1) # normalise output
scaler = MinMaxScaler(feature_range=(0, 1))
target = scaler.fit_transform(target_reshaped).flatten()

# Define our parameters
input_size = 6        # 6 steps input
output_size = 5       # 5 steps output
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_steps = [[] for _ in range(output_size)]
mae_steps = [[] for _ in range(output_size)]
mape_steps = [[] for _ in range(output_size)]
train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_size)]
train_mae_steps = [[] for _ in range(output_size)]
train_mape_steps = [[] for _ in range(output_size)]

for exp in range(num_experiments):
    X_train, X_test, y_train, y_test = split_data(data, input_size, output_size, train_ratio, seed)
    train_dataset = Time_Series_Dataset(X_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False) # changing batch size affect model accuracy significantly
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False) 
    
    # Example usage:
    hidden_sizes = [20]  # Hidden size for the LSTM layer
    num_layers = 1  # One LSTM layer
    num_filters = 64  # Number of filters for Conv1D layer
    kernel_size = 2  # Kernel size for Conv1D layer
    
    model = ConvLSTM(input_size, hidden_sizes, num_layers, output_size, num_filters, kernel_size)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # Training loop
    num_epochs = 100 # default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for inputs, targets in train_dataloader:
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
        # if epoch == 1 or epoch % 10 == 0:
            # print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.6f}')

    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for inputs, targets in test_dataloader:
            outputs = model(inputs)
            y_pred.append(outputs.numpy())
            y_test.append(targets.numpy())
    
    # Convert lists to numpy arrays
    y_pred = np.concatenate(y_pred, axis=0)
    y_test = np.concatenate(y_test, axis=0)

    mse = mean_squared_error(y_test, y_pred)
    rmse.append(np.sqrt(mse))
    
    for step in range(output_size):
        mse_step = mean_squared_error(y_pred[:, step], y_test[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))

    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []
    
    with torch.no_grad():
        for inputs, targets in train_dataloader:
            outputs = model(inputs)
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.numpy())
    
    # Convert lists to numpy arrays
    y_train_pred = np.concatenate(y_train_pred, axis=0)
    y_train_actual = np.concatenate(y_train_actual, axis=0)

    train_mse = mean_squared_error(y_train_actual, y_train_pred)
    train_rmse.append(np.sqrt(train_mse))
    
    for step in range(output_size):
        train_mse_step = mean_squared_error(y_train_pred[:, step], y_train_actual[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform for test data
    predicted_values = scaler.inverse_transform(y_pred)
    actual_values = scaler.inverse_transform(y_test)
    
    mae.append(mean_absolute_error(actual_values, predicted_values))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_values))
    
    actual_values_steps = list(zip(*actual_values))
    predicted_values_steps = list(zip(*predicted_values))
    
    for step in range(output_size):
        mae_steps[step].append(mean_absolute_error(actual_values_steps[step], predicted_values_steps[step]))
        mape_steps[step].append(mean_absolute_percentage_error(actual_values_steps[step], predicted_values_steps[step]))

    # Inverse Transform for train data
    predicted_values_train = scaler.inverse_transform(y_train_pred)
    actual_values_train = scaler.inverse_transform(y_train_actual)
    
    train_mae.append(mean_absolute_error(actual_values_train, predicted_values_train))
    train_mape.append(mean_absolute_percentage_error(actual_values_train, predicted_values_train))
    
    actual_values_train_steps = list(zip(*actual_values_train))
    predicted_values_train_steps = list(zip(*predicted_values_train))
    
    for step in range(output_size):
        train_mae_steps[step].append(mean_absolute_error(actual_values_train_steps[step], predicted_values_train_steps[step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(actual_values_train_steps[step], predicted_values_train_steps[step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

print(f"Bitcoin Conv-LSTM Regression: After {num_experiments} experimental runs, here are the results:")
print(f"Across {output_size} predictive time steps, " +
      f"Avg RMSE: {np.mean(rmse):.4f} {pm} {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_size):
    print(
        f"At time step {step + 1}, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

print("--------------------------------------------")

print(f"Bitcoin Conv-LSTM Regression on Train Data: After {num_experiments} experimental runs, here are the results:")
print(f"Across {output_size} predictive time steps, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_size):
    print(
        f"At time step {step + 1}, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%"
    )


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done
Bitcoin Conv-LSTM Regression: After 30 experimental runs, here are the results:
Across 5 predictive time steps, Avg RMSE: 0.0204 ± 0.0029, Avg MAE: 34.77 ± 3.92, Avg MAPE: 83.198% ± 45.994%
At time step 1, Avg RMSE: 0.0134 ± 0.0023, Avg MAE: 24.08 ± 4.23, Avg MAPE: 70.772% ± 44.549%
At time step 2, Avg RMSE: 0.0174 ± 0.0025, Avg MAE: 30.50 ± 3.78,

In [2]:
pd.DataFrame(actual_values).to_csv('ethereum_multi_clstm_classic_actual.csv')
pd.DataFrame(predicted_values).to_csv('ethereum_multi_clstm_classic_pred.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('ethereum_multi_clstm_classic_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('ethereum_multi_clstm_classic_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('ethereum_multi_clstm_classic_mape.csv')